In [ ]:
%pylab inline

import os
import seaborn as sns
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics

In [ ]:
import sys
sys.path.append('../../code/scripts')
#import pilot_sample_experiment_augment as ase
import utils
from importlib import reload
reload(utils)

# Plot results for different HP configs

In [ ]:
metric = 'acc'

# ERM HP Search
## Plot

In [ ]:
import plotting
reload(plotting)

# todo: should be 5 once all results are in
num_seeds = 5

lrs = [0.01,0.001,0.0001]
wds = [0.01,0.001,0.0001]
momentums = [0.9]
num_epochs = 20
obj = 'ERM'

group_id_dict = {0: 'vehicle', 1: 'animal'}


fig, ax = plt.subplots(len(lrs), len(wds), figsize=(16,16), sharex=True, sharey=True)

for i,lr in enumerate(lrs):
    for j,wd in enumerate(wds):
        for mom in momentums:
            r_ERM = utils.read_in_results(group_key='animal',
                                        results_type='subset',
                                        results_identifier='cifar4_subsetting_hpSel_< experiment name >_ERM',
                                        obj=obj,
                                        num_seeds = num_seeds,
                                        sgd_params = {'lr': lr, 
                                                      'weight_decay': wd, 
                                                      'momentum': mom},
                                        num_epochs=num_epochs)
            
            groups_ERM, subset_sizes_ERM, accs_dict_all_ERM, accs_dict_per_group_ERM = r_ERM

            plotting.plot_by_group(accs_dict_per_group_ERM,
                                   accs_dict_all_ERM,
                                   subset_sizes_ERM / subset_sizes_ERM.sum(axis=0),
                                   metric,
                                   group_id_dict,
                                   label_append = ' (ERM)',
                                   title = 'lr = {0}; wd = {1}'.format(lr,wd),
                                   ax = ax[i,j],
                                   ylim=[0.5,1]
                                  )
            
plt.suptitle('predict air/land - CV hyperparameter results for ERM objective');

## ERM - report best hp combo by subset_frac

In [ ]:
#1. aggregate results by group and hps, put in results_by_hps 
results_by_hps = pd.DataFrame(columns=['group','subset_frac','lr', 'wd', 'mom','acc','auc_roc'])

#subset_fracs = (subset_sizes/subset_sizes.sum(0))[0]
subset_fracs = [0.02, 0.05, 0.2, 0.5, 0.8, 0.95, 0.98]
for i,lr in enumerate(lrs):
    for j,wd in enumerate(wds):
        for mom in momentums:
            r_ERM = utils.read_in_results(group_key='animal',
                                        results_type='subset',
                                        results_identifier='cifar4_subsetting_hpSel_< experiment name >_ERM',
                                        obj=obj,
                                        num_seeds = num_seeds,
                                        sgd_params = {'lr': lr, 
                                                      'weight_decay': wd, 
                                                      'momentum': mom},
                                        num_epochs=num_epochs)
            
            groups, subset_sizes, accs_dict_all, accs_dict_per_group = r_ERM
            
            for g,group in enumerate(groups + ['pop',]):
                for i,subset_frac in enumerate(subset_fracs):
                    this_res_dict = {'group':g, 
                                     'subset_frac': subset_frac,
                                     'lr': lr,
                                     'wd': wd,
                                     'mom': mom,
                                     }
                    
                    if group == 'pop':
                        for acc_key in ['acc', 'auc_roc']:
                            avg_acc = np.mean(accs_dict_all[acc_key][i])
                            this_res_dict[acc_key] = avg_acc
                    
                    else:
                        for acc_key in ['acc', 'auc_roc']:
                            avg_acc = np.mean(accs_dict_per_group[acc_key][g][i])
                            this_res_dict[acc_key] = avg_acc
            
                    results_by_hps = results_by_hps.append(this_res_dict, ignore_index=True)
     

In [ ]:
# Find argmax per (group, subset_frac) pair
res_grouped = results_by_hps.groupby(['group','subset_frac'])
idxs_maxes = res_grouped[metric].idxmax()

# aggregate results of the optimal for each pair, along with their accuracies
op_df = pd.DataFrame()
for g, group in enumerate(groups + ['pop',]):
    for subset_frac in subset_fracs:
        idx_max = idxs_maxes[g,subset_frac]
        
        op_df =op_df.append(results_by_hps.iloc[idx_max])

In [ ]:
op_df

# IS HP Search
## Plot

In [ ]:
import plotting
reload(plotting)

# todo: should be 5 once all results are in
num_seeds = 5

lrs = [0.01,0.001,0.0001]
wds = [0.01,0.001,0.0001]
momentums = [0.9]
num_epochs = 20
obj = 'IS'

group_id_dict = {0: 'vehicle', 1: 'animal'}


fig, ax = plt.subplots(len(lrs), len(wds), figsize=(16,16), sharex=True, sharey=True)

for i,lr in enumerate(lrs):
    for j,wd in enumerate(wds):
        for mom in momentums:
            r_ERM = utils.read_in_results(group_key='animal',
                                        results_type='subset',
                                        results_identifier='cifar4_subsetting_hpSel_< experiment name >_IS',
                                        obj=obj,
                                        num_seeds = num_seeds,
                                        sgd_params = {'lr': lr, 
                                                      'weight_decay': wd, 
                                                      'momentum': mom},
                                        num_epochs=num_epochs)
            
            groups_ERM, subset_sizes_ERM, accs_dict_all_ERM, accs_dict_per_group_ERM = r_ERM

            plotting.plot_by_group(accs_dict_per_group_ERM,
                                   accs_dict_all_ERM,
                                   subset_sizes_ERM / subset_sizes_ERM.sum(axis=0),
                                   metric,
                                   group_id_dict,
                                   label_append = ' (IS)',
                                   title = 'lr = {0}; wd = {1}'.format(lr,wd),
                                   ax = ax[i,j],
                                   ylim=[0.5,1]
                                  )
            
plt.suptitle('predict air/land - CV hyperparameter results for IS objective');

## IS - report best hp combo by subset_frac

In [ ]:
#1. aggregate results by group and hps, put in results_by_hps 
results_by_hps = pd.DataFrame(columns=['group','subset_frac','lr', 'wd', 'mom','acc','auc_roc'])

subset_fracs = [0.02, 0.05, 0.2, 0.5, 0.8, 0.95, 0.98]
for i,lr in enumerate(lrs):
    for j,wd in enumerate(wds):
        for mom in momentums:
            r_ERM = utils.read_in_results(group_key='animal',
                                        results_type='subset',
                                        results_identifier='cifar4_subsetting_hpSel_< experiment name >_IS',
                                        obj=obj,
                                        num_seeds = num_seeds,
                                        sgd_params = {'lr': lr, 
                                                      'weight_decay': wd, 
                                                      'momentum': mom},
                                        num_epochs=num_epochs)
            
            groups, subset_sizes, accs_dict_all, accs_dict_per_group = r_ERM
            
            for g,group in enumerate(groups + ['pop',]):
                for i,subset_frac in enumerate(subset_fracs):
                    this_res_dict = {'group':g, 
                                     'subset_frac': subset_frac,
                                     'lr': lr,
                                     'wd': wd,
                                     'mom': mom,
                                     }
                    
                    if group == 'pop':
                        for acc_key in ['acc', 'auc_roc']:
                            avg_acc = np.mean(accs_dict_all[acc_key][i])
                            this_res_dict[acc_key] = avg_acc
                    
                    else:
                        for acc_key in ['acc', 'auc_roc']:
                            avg_acc = np.mean(accs_dict_per_group[acc_key][g][i])
                            this_res_dict[acc_key] = avg_acc
            
                    results_by_hps = results_by_hps.append(this_res_dict, ignore_index=True)
     

In [ ]:
# Find argmax per (group, subset_frac) pair
res_grouped = results_by_hps.groupby(['group','subset_frac'])
idxs_maxes = res_grouped[metric].idxmax()

# aggregate results of the optimal for each pair, along with their accuracies
op_df = pd.DataFrame()
for g, group in enumerate(groups + ['pop',]):
    for subset_frac in subset_fracs:
        idx_max = idxs_maxes[g,subset_frac]
        
        op_df =op_df.append(results_by_hps.iloc[idx_max])

In [ ]:
# print this. Each row corresponds to the optimal CV results for the given (subset_frac/group column)
# group = 2.0 means population -- this does argumax on metric (either 'acc' or 'auc_roc')
op_df

# GDRO HP Search
## Plot

In [ ]:
import plotting
reload(plotting)

# todo: should be 5 once all results are in
num_seeds = 5

lrs = [0.001]
wds = [0.001, 0.0001]
momentums = [0.9]
num_epochs = 20
obj = 'GDRO'
group_adjs = [1.0, 4.0, 8.0]
gdro_ss = [1e-1, 1e-2, 1e-3]

group_id_dict = {0: 'vehicle', 1: 'animal'}


fig1, ax1 = plt.subplots(len(group_adjs), len(gdro_ss), figsize=(16,16), sharex=True, sharey=True)

fig2, ax2 = plt.subplots(len(group_adjs), len(gdro_ss), figsize=(16,16), sharex=True, sharey=True)

figs = [fig1, fig2]
axs = [ax1, ax2]

for lr in lrs:
    for k,wd in enumerate(wds):
        fig = figs[k]
        ax = axs[k]
        for mom in momentums:
            for i,ga in enumerate(group_adjs):
                for j,ss in enumerate(gdro_ss):
                    r_ERM = utils.read_in_results(group_key='animal',
                                                results_type='subset',
                                                results_identifier='cifar4_subsetting_hpSel_< experiment name >_GDRO',
                                                obj=obj,
                                                num_seeds = num_seeds,
                                                sgd_params = {'lr': lr, 
                                                              'weight_decay': wd, 
                                                              'momentum': mom,
                                                              'GDRO_group_adj': ga,
                                                              'gdro_stepsize': ss},
                                                num_epochs=20)

                    groups_ERM, subset_sizes_ERM, accs_dict_all_ERM, accs_dict_per_group_ERM = r_ERM

                    plotting.plot_by_group(accs_dict_per_group_ERM,
                                           accs_dict_all_ERM,
                                           subset_sizes_ERM / subset_sizes_ERM.sum(axis=0),
                                           metric,
                                           group_id_dict,
                                           label_append = ' (GDRO)',
                                           title = 'wd = {1}; gdro_ga = {2}; gdro_ss = {3}'.format(lr,wd,ga,ss),
                                           ax = ax[i,j],
                                           ylim=[0.8 ,1]
                                          )

plt.suptitle('predict animal/vehicle - CV hyperparameter results for GDRO objective');



## GDRO - report best hp combo by subset_frac

In [ ]:
#1. aggregate results by group and hps, put in results_by_hps 
results_by_hps = pd.DataFrame(columns=['group','subset_frac','lr', 'wd', 'mom','acc','auc_roc'])

#subset_fracs = (subset_sizes/subset_sizes.sum(0))[0]
subset_fracs = [0.02, 0.05, 0.2, 0.5, 0.8, 0.95, 0.98]
for i,lr in enumerate(lrs):
    for j,wd in enumerate(wds):
        for k, ga in enumerate(group_adjs):
            for m, ss in enumerate(gdro_ss):
                for mom in momentums:
                    r_ERM = utils.read_in_results(group_key='animal',
                                                results_type='subset',
                                                results_identifier='cifar4_subsetting_hpSel_< experiment name >_GDRO',
                                                obj=obj,
                                                num_seeds = num_seeds,
                                                sgd_params = {'lr': lr, 
                                                              'weight_decay': wd, 
                                                              'momentum': mom,
                                                              'GDRO_group_adj': ga,
                                                              'gdro_stepsize': ss},
                                                num_epochs=num_epochs)

                    groups, subset_sizes, accs_dict_all, accs_dict_per_group = r_ERM

                    for g,group in enumerate(groups + ['pop',]):
                        for i,subset_frac in enumerate(subset_fracs):
                            this_res_dict = {'group':g, 
                                             'subset_frac': subset_frac,
                                             'lr': lr,
                                             'wd': wd,
                                             'mom': mom,
                                             'group_adj': ga,
                                             'gdro_ss': ss
                                             }

                            if group == 'pop':
                                for acc_key in ['acc', 'auc_roc']:
                                    avg_acc = np.mean(accs_dict_all[acc_key][i])
                                    this_res_dict[acc_key] = avg_acc

                            else:
                                for acc_key in ['acc', 'auc_roc']:
                                    avg_acc = np.mean(accs_dict_per_group[acc_key][g][i])
                                    this_res_dict[acc_key] = avg_acc

                            results_by_hps = results_by_hps.append(this_res_dict, ignore_index=True)


In [ ]:
# Find argmax per (group, subset_frac) pair
res_grouped = results_by_hps.groupby(['group','subset_frac'])
idxs_maxes = res_grouped[metric].idxmax()
groups = ['animal', 'vehicle']
# aggregate results of the optimal for each pair, along with their accuracies
op_df = pd.DataFrame()
for g, group in enumerate(groups + ['pop',]):
    for subset_frac in subset_fracs:
        idx_max = idxs_maxes[g,subset_frac]
        
        op_df =op_df.append(results_by_hps.iloc[idx_max])

In [ ]:
op_df

In [ ]:
# Find which group has the lowest accuracy for each subset_frac & parameter setting
res_grouped1 = results_by_hps.groupby(['subset_frac', 'lr', 'wd', 'mom', 'gdro_ss', 'group_adj'])
idxs_mins = res_grouped1[metric].idxmin()

gdro_mins_df = pd.DataFrame()
for frac in subset_fracs:
    for lr in lrs:
        for mom in momentums:
            for wd in wds:
                for ga in group_adjs:
                    for ss in gdro_ss:
                        idx_min = idxs_mins[frac, lr, wd, mom, ss, ga]
                        gdro_mins_df = gdro_mins_df.append(results_by_hps.iloc[idx_min])

# Find which hyperparameters lead to the highest accuracy for each subset (out of the lowest accuracies from above)
res_grouped2 = gdro_mins_df.groupby(['subset_frac'])
idxs_maxs = res_grouped2[metric].idxmax()
gdro_opt_df = pd.DataFrame()

for frac in subset_fracs:
    idx_max = idxs_maxs[frac]
    gdro_opt_df = gdro_opt_df.append(results_by_hps.iloc[idx_max])
    

In [ ]:
gdro_opt_df